# 394. Decode String
Given an encoded string, return its decoded string.

Encoding rule:
`k[encoded_string]` where the `encoded_string` in the brackets should be repeated exactly
`k` times.

Encoded messages can be nested.

Assumptions:
- `k` will always be positive
- The brackets will always close correctly
- All of the encoded messages will always be correct without any errors

Examples:
- `s = "3[a]2[bc]"` -> `"aaabcbc"`
- `s = "3[a2[c]]"` -> `"accaccacc"`

**Timing**\
**Start: 16:58**\
**End: 17:30**

### Planning
**17:01**\
I could solve this recursively, or I could iterate over the encoded message
and build substrings with a helper function.

One thing to consider is how to separate or identify the sub-messages. I think that
it might be more effective to iterate over the entire message and break out to the helper
function whenever numbers are encountered?

Here is the outline for what I'll do:
1. Define helper function
2. Instantiate empty decoded_string object
3. Iterate over the encoded message in a while loop with starting index of 0
4. At each iteration, check:
     - is the current value a letter? if so add to index
     - is the current value a number? if so call helper function with `i` to decrypt the encountered unit

`decode_substring` function:
- Will need to keep track of the opened brackets
- Will need to keep track of the substring
- Should return the substring multiplied by the input value
- Should return the updated i and the resulting substring

Steps:
1. add 1 to opened_brackets, define an empty substring, record current decode_scalar
2. iterate over string at current index, check current message[i] value:
    - if a letter, add it to the substring
    - if a number, call the function again and add the resulting substring to the current
    substring
    - if a closed bracket, subtract one from opened bracket
3. At the end of each function, if the `opened_brackets` is 0, break the loop, return i and substring*decode_scalar


In [17]:
def decode_string(s):
    def decode_scalar(current_index):
        scalar = ""
        while True:
            if s[current_index].isnumeric():
                scalar += s[current_index]
            else:
                break
            current_index += 1
                
        return current_index+1, int(scalar)
    
    def decode_substring(current_index):
        """Returns updated index, decoded substring, and current opened brackets."""
        current_index, scalar = decode_scalar(current_index)
        opened_brackets = True
        substring = ""
        
        while current_index < len(s):
            if s[current_index].isalpha():
                substring += s[current_index]
            elif s[current_index] == "]":
                opened_brackets = False
            elif s[current_index].isnumeric():
                current_index, new_substring = decode_substring(current_index)
                substring += new_substring
            
            if not opened_brackets:
                break
            current_index += 1
        
        return current_index, substring*scalar
    
    decoded_message = ""
    i = 0
    while i < len(s):
        if s[i].isalpha():
            decoded_message += s[i]
        elif s[i].isnumeric():
            i, substring = decode_substring(i)
            decoded_message += substring
        i += 1
    
    return decoded_message

s = "3[a2[c]]"
decode_string(s)

'accaccacc'

My initial solution worked in the majority of use-cases, but it did not work for multi-integer numbers.

I'm going to replace this line:
`decode_scalar = int(s[current_index])` with a `decode_scalar` function.

### Afterthoughts
**Stats:**
Runtime
25 ms
Beats
70.4%
Memory
13.8 MB
Beats
7.40%

The memory usage in this implementation was abysmal, but the speed was pretty good.

I'm going to check out some other implementations to see how they might have solved it.

Someone has presented much simpler solution using a `stack`. 

The stack basically keeps a record of previous strings and previous numbers.

In [28]:
def decode_string_stack(s):
    stack = []
    current_scalar = 0
    decoded_string = ""
    
    for c in s:
        if c == "[":
            stack.append(decoded_string)
            stack.append(current_scalar)
            current_scalar = 0
            decoded_string = ""
        
        elif c == "]":
            prev_scalar = stack.pop()
            prev_string = stack.pop()
            decoded_string = prev_string + decoded_string*prev_scalar
            
        elif c.isnumeric():
            current_scalar = current_scalar * 10 + int(c)
            
        else:
            decoded_string += c
    
    return decoded_string

s = "2[a2[c2[b]]]"
decode_string_stack(s)

'acbbcbbacbbcbb'

I like the elegance of this solution, but the interesting thing about this is that the stats are almost identical to my original solution.